In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px

In [8]:
df = pd.read_csv(os.path.join("..","data/raw", "FAO.csv"), encoding='ISO-8859-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21477 entries, 0 to 21476
Data columns (total 63 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area Abbreviation  21477 non-null  object 
 1   Area Code          21477 non-null  int64  
 2   Area               21477 non-null  object 
 3   Item Code          21477 non-null  int64  
 4   Item               21477 non-null  object 
 5   Element Code       21477 non-null  int64  
 6   Element            21477 non-null  object 
 7   Unit               21477 non-null  object 
 8   latitude           21477 non-null  float64
 9   longitude          21477 non-null  float64
 10  Y1961              17938 non-null  float64
 11  Y1962              17938 non-null  float64
 12  Y1963              17938 non-null  float64
 13  Y1964              17938 non-null  float64
 14  Y1965              17938 non-null  float64
 15  Y1966              17938 non-null  float64
 16  Y1967              179

# Découverte
Nous avons un dataset sur la production de denrées dans le monde.

On apprend donc les informations de bases sur ce dataset, compris avec **63 attributs** et **21477 observations**
On peut séparer ces attributs en 2 partie : les **attributs sur le produit** et les **attributs temporels**
On constate d'ailleurs que les valeurs manquantes sont uniquement présentes dans les attributs temporels, qui pourrait traduire simplement l'abscence de production (à explorer)


In [9]:
df.head(3)

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360


# Organisation

Le dataset est rangé par pays, puis chaque ligne correspond à un produit. On y associe les informations sur le pays (position, abréviation,...) et les informations sur le produit (nom, type de produit) surtout avec sa production par année (Y2004, Y2005, Y....)  
Avant de combler les valeurs vides, on vérifie si on peut enlever des colonnes inutile : Unit possède vraissemblablement la même valeur (à vérifier), abréviation et le code ne nous sont pas utiles, l'élément et le produit code non plus et toute les colonnes Y... pourrait n'en former qu'une (après avoir gérer les valeurs nuls)

In [10]:
print(df["Unit"].unique())

['1000 tonnes']


In [12]:
dropped_cols = ["Area Abbreviation", "Area Code", "Item Code", "Element Code", "Unit"]
df.drop(dropped_cols, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21477 entries, 0 to 21476
Data columns (total 58 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       21477 non-null  object 
 1   Item       21477 non-null  object 
 2   Element    21477 non-null  object 
 3   latitude   21477 non-null  float64
 4   longitude  21477 non-null  float64
 5   Y1961      17938 non-null  float64
 6   Y1962      17938 non-null  float64
 7   Y1963      17938 non-null  float64
 8   Y1964      17938 non-null  float64
 9   Y1965      17938 non-null  float64
 10  Y1966      17938 non-null  float64
 11  Y1967      17938 non-null  float64
 12  Y1968      17938 non-null  float64
 13  Y1969      17938 non-null  float64
 14  Y1970      17938 non-null  float64
 15  Y1971      17938 non-null  float64
 16  Y1972      17938 non-null  float64
 17  Y1973      17938 non-null  float64
 18  Y1974      17938 non-null  float64
 19  Y1975      17938 non-null  float64
 20  Y1976 

# Valeurs nuls
Maintenant avant de créer la colonne regroupant toute les années, il faut combler les valeurs nuls.  
Pour ça plusieurs choix : la médiane, la moyenne, le mode ou une valeur par défaut (0, 1, 36, ...).  
On constate que plus l'année est vieille, plus il y a de valeur nul. Ainsi on peut savoir que les valeurs nuls correspondent au moment avant la production (ou avant qu'on trace la production). Donc on peut se demander comment combler les valeurs nuls ?  
La médiane et la moyenne ne sont pas sencé, avec une évolution ce serait bizarre de commencer par la médiane puis de redescendre...  
Le mode subit la même problématique.  
Enfin il parraît logique de combler les valeur vides avec 0, pour montrer l'absence de production.

In [13]:
df.fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21477 entries, 0 to 21476
Data columns (total 58 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       21477 non-null  object 
 1   Item       21477 non-null  object 
 2   Element    21477 non-null  object 
 3   latitude   21477 non-null  float64
 4   longitude  21477 non-null  float64
 5   Y1961      21477 non-null  float64
 6   Y1962      21477 non-null  float64
 7   Y1963      21477 non-null  float64
 8   Y1964      21477 non-null  float64
 9   Y1965      21477 non-null  float64
 10  Y1966      21477 non-null  float64
 11  Y1967      21477 non-null  float64
 12  Y1968      21477 non-null  float64
 13  Y1969      21477 non-null  float64
 14  Y1970      21477 non-null  float64
 15  Y1971      21477 non-null  float64
 16  Y1972      21477 non-null  float64
 17  Y1973      21477 non-null  float64
 18  Y1974      21477 non-null  float64
 19  Y1975      21477 non-null  float64
 20  Y1976 

Maintenant nous pouvons créer cette colonne.

In [18]:
df["years_evolution"] = [df["Y"+str(i)] for i in range(1966,2014)]
df.head(3)

1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012


ValueError: Length of values (48) does not match length of index (21477)